# [0] Imports

In [764]:
import Pkg;

using LinearAlgebra, Random, Gurobi, JuMP, Distributions, Plots, LazySets

using Dates

# [1] Setup

## [1.1] Parameters

In [765]:
n_jobs = 25
n_vehicles_jobs = n_jobs
n_vehicles_coverage = 10
T = 2 * n_jobs
min_duration = 2
max_duration = 6
speed = 1000/6
coverage_distance = 50
size = 500
step = 50;

## [1.2] Locations, Windows, Loads

In [766]:
Random.seed!(1234)

function create_cluster_sizes(jobs_total)
    Random.seed!(1234)
    jobs_created = 0
    cluster = []
    min_jobs_allowed = n_jobs >= 15 ? 3 : 2
        target_clusters = 5
        while jobs_created != jobs_total
            num_to_add = rand(min(min_jobs_allowed, jobs_total - jobs_created) : 
                min(Int((n_jobs/target_clusters)÷1), jobs_total - jobs_created))
            jobs_created += num_to_add
            push!(cluster, num_to_add)
        end
    
    return cluster
end;

In [767]:
time_windows = []
locations = rand(Uniform(0,size), 1, 2)
work_load = []
cluster = create_cluster_sizes(n_jobs);

In [768]:
function create_time_windows_and_work_load(cluster_sizes, locations)
    Random.seed!(1234)
    locations = rand(Uniform(0,size), 1, 2)
    for size_c in cluster
        first = rand(Uniform(0,size), 1, 2)
        locations = vcat(locations, first)

        job_begins = rand(2:10)
        job_finish = rand((job_begins+min_duration):(job_begins+max_duration))
        push!(time_windows, [job_begins, job_finish])

        time_work = rand(min_duration:max(min_duration, job_finish - job_begins))
        push!(work_load, time_work)

        for neighbour in 1:(size_c-1)
            new_x = rand(Uniform(max(0,first[1]-20), min(first[1]+20, size)), 1, 1)
            new_y = rand(Uniform(max(0,first[2]-20), min(first[2]+20, size)), 1, 1)
            new = hcat(new_x, new_y)
            locations = vcat(locations, new)

            job_begins = rand(job_finish:min(T-min_duration-2, job_finish + 6))
            job_finish = rand((job_begins+min_duration):(min(job_begins+max_duration, T-2)))
            push!(time_windows, [job_begins, job_finish])

            time_work = rand(min_duration:min(max_duration, job_finish-job_begins))
            push!(work_load, time_work)
        end
    end
    
    return [time_windows, work_load, locations]
end;

In [769]:
time_windows, work_load, locations = create_time_windows_and_work_load(cluster, locations)
distances = [LinearAlgebra.norm(locations[i, :] .- locations[j, :]) for i=1:n_jobs+1, j = 1:n_jobs+1];

# [2] Helper Methods

## 2.1 Get initial routes

In [770]:
routes = []

for n in 1:n_jobs
    job_route = []
    dist = distances[1, n+1]

    min_t = Int(floor(time_windows[n][1] - dist/speed))
    max_t = Int(ceil(time_windows[n][2] + dist/speed))

    push!(job_route, [[0, min_t], [n, time_windows[n][1]], dist])
    
    for t in time_windows[n][1]:(time_windows[n][2]-1)
        push!(job_route, [[n, t], [n, t+1], 0])
    end
    
    push!(job_route, [[n, time_windows[n][2]], [n_jobs+1, max_t],  dist])
    push!(routes, job_route)
    
end

## 2.2 Compute Cost

In [771]:
function compute_cost(route)
    cost = 0
    for entity in route
        cost += entity[3]
    end
    return cost
end;

In [772]:
C = []
for i in 1:length(routes)
    push!(C, compute_cost(routes[i]))
end

## 2.3 Compute Delta

In [773]:
function compute_delta(routes)
    Q = length(routes)
    delta = [[[0.0 for q in 1:Q] for t in 1:T] for i in 1:n_jobs]
    for rindex in 1:Q
        route = routes[rindex]
        for arc in route
            loc1, time1 = arc[1]
            loc2, time2 = arc[2]
            if (loc1 != 0) & (loc1 != n_jobs + 1)
                delta[loc1][time1][rindex] = 1
            end
            
            if (loc2 != 0) & (loc2 != n_jobs + 1)
                delta[loc2][time2][rindex] = 1
            end
        end
    end
    return delta
end;

In [774]:
delta = compute_delta(routes);

## 2.4 Compute U

In [775]:
function compute_u(routes)
    Q = length(routes)
    u = [[0 for q in 1:Q] for i in 1:n_jobs]
    
    for rindex in 1:Q
        route = routes[rindex]
        for arc in route
            loc1 = arc[1][1]
            loc2 = arc[2][1]
            if (1 <= loc1 <= n_jobs)
                u[loc1][rindex] = 1
            end
            if (1 <= loc2 <= n_jobs)
                u[loc2][rindex] = 1
            end
        end
    end
    return u
end;

In [776]:
u = compute_u(routes);

## 2.5 Label-Setting Algorithm

In [777]:
function sp_lsa(n, travel_distance, travel_time, windows, load, rho_v, pi_v, mu_v)
    N = [[1]] 
    Times = [ [0] ] 
    R = [rho_v]  
    L = [1]  

    current_state = 1
    total_state = 1

    #=
    NOTE: For pi_v, u_v, mu_v, and delta_v, there is no defined value
    for the actual depot. We will be using -1's for indexing quite liberally,
    as in our loop, 1 = n+2 = depot, when in 'reality', this should be 0 = n+!.

    This is also the case for variables windows and load. This happened last LSA too.
    =#
    while 1==1

        # STEP 1: Check whether we want to move on to checking the next path
        if (L[current_state] == n+2)
            current_state += 1
            if current_state > total_state
                # print(current_state, " ", total_state, " nothing more to check")
                break
            else
                continue
            end
        end

        # STEP 2: Enter the for loop.
        for i in 2:(n+2)

            # STEP 2-1: Check that the current node is not already inside our path.
            if ~(i in N[current_state])

                #= STEP 2-2: Feasibility check. We must have enough time
                to go from our current location (end time T[current_state]), plus
                the travel time from that end node to the new node (travel_time[L[current_state], i]),
                plus the amount of time it takes to work the OLD job (load[L[current_state]-1]),
                plus the amount of time it takes to work the NEW job (load[i-1])

                this must be smaller than or equal to the end time of the window for the NEW job (windows[i-1][2])
                =#
                cur_time = last(Times[current_state])
                dist_nec = travel_time[L[current_state], i]
                old_job_time_nec = L[current_state] > 1 ? load[L[current_state]-1] : 0
                new_job_time_nec = load[i-1]
                new_window_start, new_window_close = windows[i-1]

                if L[current_state] != 1
                    if cur_time + dist_nec + old_job_time_nec + new_job_time_nec > new_window_close
                        continue
                    end
                end

                reach_new_node_time = max(cur_time + dist_nec + old_job_time_nec, new_window_start)

                #=
                The topic of mu_it needs to be discussed.

                There are two competing theories. Either mu_it should only be added
                when we are at the node for the first time, or it should be added 
                every time we are at the node, even if we're staying and haven't just hit it.

                It appears that if we only subtract mu once when we hit, there will be subtle
                but noticeable errors in our final answer. They are practically slight, but 
                the answer is still wrong.

                So I'm thinking that if we fix the mu issue, everything will be resolved.
                =#

                #= When we hit a node, we should account for the mu accumulated THROUGHOUT
                staying at node i. This makes more conceptual sense than "retroactively"
                fitting old values of mu. It also means we are forced to look ahead.

                First, we determine when we first arrive at the node. This is indicated above
                as reach_new_node_time. Then, we will be at the node for the following times:
                reach_new_node_time, reach_new_node_time+1, ..., reach_new_node_time + load[i-1]

                So we will need to patch together the mu for ALL of these times!
                =#

                # SECOND CHECK
                add_segdist = travel_distance[L[current_state], i]
                subtract_pi = i < n+2 ? -pi_v[i-1] : 0
                subtract_mu = 0.0
                if i < n+2
                    for applicable_time in reach_new_node_time:reach_new_node_time+new_job_time_nec
                        subtract_mu -= mu_v[Int(i-1), Int(applicable_time)]
                    end
                end

                proposed_cost = R[current_state] + add_segdist + subtract_pi + subtract_mu

                # STEP 2-3: Add the path to our N.

                # First, copy the original state so we can modify it
                push!(N, copy(N[current_state]))
                # Add this new, guaranteed-to-be-valid, node.
                push!(N[total_state+1], i)

                # Time is going to be when we GET to the final node.

                #= I argue that we also need to include a work load term into the 
                term bounded by current_time + necessary_travel_distance.

                This is because there's no point in going to an old place, not 
                doing any work, and then going away to some new place.

                So in reality the check becomes tripartite.
                =#
                new_times_array = copy(Times[current_state])
                push!(new_times_array, reach_new_node_time)
                push!(Times, new_times_array)

                # Cost.
                push!(R, proposed_cost)

                # Last node is updated.
                push!(L, i)

                total_state += 1

            end
        end

        current_state += 1 # concluded iteration for that state in mind. Now next one

        #= It's possible that we've just caught up to the final total state, meaning we didn't generate
        anything and can stop here. =#
        if current_state == total_state
            # print(current_state, " ", total_state, " came from bottom")
            break
        end     
    end

    return N, R, Times
end;

### 2.5.1 Label-Setting Algorithm Constant Variables

In [778]:
distances_label = deepcopy(distances)
distances_label = hcat(distances_label, distances_label[:, 1])
distances_label = vcat(distances_label, collect(push!(distances_label[:, 1], 0)'));

In [779]:
travel_times = ceil.(distances_label / speed);

In [780]:
windows_label = deepcopy(time_windows)
push!(windows_label, [0, 100]);

In [781]:
load_label = deepcopy(work_load)
push!(load_label, 0);

## 2.6 Extract Best Route in SP

In [782]:
function extract_best_route(sub_paths, sub_rc, sub_times, n_jobs)

    best_rc = Inf
    best_index = 0

    for i in 1:length(sub_paths)
        if sub_rc[i] < best_rc && last(sub_paths[i]) == n_jobs + 2
            best_rc = sub_rc[i]
            best_index = i
        end
    end

    return sub_paths[best_index], sub_rc[best_index], sub_times[best_index]
end;

## 2.7 Generate full time info on route

In [783]:
function generate_full_time_info(best_route, best_times)
    best_route_info = []
    for locindex in 1:length(best_route)-1
        #=
        Strategy:
        If you are not at a job location, you don't need to capture
        stationary information. Otherwise you do.

        Anyways we want data moving from one to the next.
        =#
        cur_loc = best_route[locindex]
        cur_time = best_times[locindex]
        time_to_start_moving = best_times[locindex]

        #stationary data comes first.
        if 1 < locindex < n_jobs + 2
            cur_job_num = cur_loc - 1 # delete padding!

            #stay stationary for however long it takes for you to do the job

            # Note: eventually this will have to be modified to include coverage.
            # Eventually we will write a subproblem-tells-us-when-work-is-done component.

            for time in cur_time:cur_time+work_load[cur_job_num]-1
                route_detail = [ [cur_loc-1, time], [cur_loc-1, time+1], 0]
                push!(best_route_info, route_detail)
            end

            time_to_start_moving += work_load[cur_job_num]
        end

        new_loc = best_route[locindex+1]
        #then if we move, we move.
        route_detail = [ [cur_loc-1, time_to_start_moving], [new_loc-1, best_times[locindex+1]], distances_label[cur_loc, new_loc]]
        push!(best_route_info, route_detail)
    end

    return best_route_info
end;

# [3] Column Generation Loop

In [784]:
modelcg = Model(Gurobi.Optimizer);
set_optimizer_attribute(modelcg, "OutputFlag", 0);

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-04


In [785]:
@variable(modelcg, 0 <= y_s[1:n_jobs, 1:T] <= 1) # should be Bin
@variable(modelcg, 0 <= y_e[1:n_jobs, 1:T] <= 1); # should be Bin

@constraint(modelcg, start[i in 1:n_jobs, t in 1:(T-1)], y_s[i,t] <= y_s[i,t+1])
@constraint(modelcg, ends[i in 1:n_jobs, t in 1:(T-1)], y_e[i,t] <= y_e[i,t+1])

@constraint(modelcg, window_start[i in 1:n_jobs, t in 1:(time_windows[i][1]-1)], y_s[i,t] == 0)
@constraint(modelcg, window_ends[i in 1:n_jobs, t in time_windows[i][2]:T], y_e[i,t] == 1)

@constraint(modelcg, duration[i in 1:n_jobs], sum(y_s[i,t] - y_e[i,t] for t in 1:T) >= work_load[i]);

@constraint(modelcg, coverage[i in 1:n_jobs, t in 1:T], y_s[i, t] - y_e[i, t] <= 1);

In [786]:
keep_going = true

num_iters = 0
num_routes_at_a_time = 1
z_values = []
OBJ = 0

while keep_going
    num_iters += 1
    Q = length(routes)
    
    unregister(modelcg, :z)
    @variable(modelcg, 0 <= z[1:Q] <= 1) # should be Bin
    
    unregister(modelcg, :unique)
    @constraint(modelcg, unique[i in 1:n_jobs], sum(u[i][q] * z[q] for q in 1:Q) >= 1);
    
    unregister(modelcg, :driver)
    @constraint(modelcg, driver, sum(z[q] for q in 1:Q) <= n_vehicles_jobs)
    
    unregister(modelcg, :work)
    @constraint(modelcg, work[i in 1:n_jobs, t in 1:(T-1)], 
        y_s[i, t] - y_e[i, t] <= sum(z[q] * delta[i][t][q] for q in 1:Q));
    
    @objective(modelcg, Min, sum(compute_cost(routes[q]) * z[q] for q in 1:Q));
    
    optimize!(modelcg)
    OBJ = objective_value(modelcg)
    
    push!(z_values, round.(value.(z), digits = 3))
    
    pi_values = dual.(unique)
    rho_value = dual.(driver)
    mu_values = dual.(work);
    
    sub_paths, sub_rc, sub_times = sp_lsa(n_jobs, distances_label, travel_times, windows_label, load_label, rho_value, pi_values, mu_values);
    
    best_route, best_rc, best_times = extract_best_route(sub_paths, sub_rc, sub_times, n_jobs)
    
    if best_rc > -1e-6
        keep_going = false
        break
    end
    
    push!(routes, generate_full_time_info(best_route, best_times))
    
    u = compute_u(routes)
    delta = compute_delta(routes);
    
    C = []
    for i in 1:length(routes)
        push!(C, compute_cost(routes[i]))
    end;
end

In [787]:
route_descs = []
for i in n_jobs+1:length(routes)
    if last(z_values)[i] > 0.1
        nodes_visited = []
        for entity in routes[i]
            place = entity[1][1]
            if ~(place in nodes_visited)
                push!(nodes_visited, place)
            end
        end
        push!(nodes_visited, n_jobs+1)
        push!(route_descs, nodes_visited)
    end
end

In [788]:
route_descs

7-element Vector{Any}:
 Any[0, 1, 2, 3, 26]
 Any[0, 24, 25, 26]
 Any[0, 8, 9, 10, 26]
 Any[0, 20, 21, 22, 23, 26]
 Any[0, 16, 17, 18, 19, 13, 14, 26]
 Any[0, 4, 5, 6, 7, 26]
 Any[0, 11, 12, 15, 26]

In [789]:
seen = []
for list in route_descs
    for elt in list
        push!(seen, elt)
    end
end
print(sort(seen))

Any[0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 26, 26, 26, 26, 26, 26]

In [790]:
print(last(z_values))

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [791]:
for job_num in 1:n_jobs
    job_included = false
    for route in route_descs
        if job_num in route
            job_included = true
        end
    end
    
    if ~job_included
        println("Job ", job_num, " was not in a fancy route.")
    end
end

In [792]:
for i in 1:length(z_values)
    #println(z_values[i], "\n")
end

In [793]:
println(OBJ)

2845.663216934919
